In [1]:
#all imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import json
import plotly.graph_objects as go

#SPOTIPY_CLIENT_ID = "5f222d8ae8c744f6815d314bba7709ab"
#SPOTIPY_CLIENT_SECRET = "2e90b0c27de84414ac5a7847d8bc73c9"

In [2]:
#set playlist + spotify authentication
playlistURL = 'https://open.spotify.com/playlist/596TVWnQdvHEeYyKUXkpDL?si=d36cdf733a8945c5'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

#get playlist items
playlist = spotify.playlist_items(playlistURL)

In [3]:
#write playlist data to JSON file
with open('data.json', 'w') as f:
    json.dump(playlist, f)
    
#read the JSON file using pandas to convert to DataFrame
data = json.load(open('data.json'))

df = pd.DataFrame(data)
df.head()


,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/596TVWnQd...,"{'added_at': '2019-05-14T05:14:13Z', 'added_by...",100,None,0,None,76
1,https://api.spotify.com/v1/playlists/596TVWnQd...,"{'added_at': '2019-05-14T05:13:56Z', 'added_by...",100,None,0,None,76
2,https://api.spotify.com/v1/playlists/596TVWnQd...,"{'added_at': '2019-05-14T05:13:12Z', 'added_by...",100,None,0,None,76
3,https://api.spotify.com/v1/playlists/596TVWnQd...,"{'added_at': '2019-05-14T05:12:48Z', 'added_by...",100,None,0,None,76
4,https://api.spotify.com/v1/playlists/596TVWnQd...,"{'added_at': '2019-05-14T05:12:12Z', 'added_by...",100,None,0,None,76


In [4]:
#initialise item ID list
IDlist = []

In [5]:
#get total songs in playlist
totalSongs = df.iloc[0]['total']

#for loop to retreive all song IDs from the playlist
for x in range(0, totalSongs): 
    pl_item = df.iloc[x]['items']
    
    #write playlist data to JSON file
    with open('data.json', 'w') as f:
        json.dump(pl_item, f)
    
    #read the JSON file using pandas to convert to DataFrame
    data3 = json.load(open('data.json'))
    #find track ID
    itemDF = pd.DataFrame(data3)
    trackID = itemDF['track'].loc['uri']
    IDlist.append(trackID)



In [6]:
#get number of IDs in IDlist
print(len(IDlist))

76


In [7]:
#create list of track features of all songs
trackFeaturesList = spotify.audio_features(tracks=IDlist)
print(trackFeaturesList)

[{'danceability': 0.342, 'energy': 0.236, 'key': 2, 'loudness': -8.276, 'mode': 1, 'speechiness': 0.0321, 'acousticness': 0.762, 'instrumentalness': 0.00374, 'liveness': 0.385, 'valence': 0.185, 'tempo': 76.747, 'type': 'audio_features', 'id': '4caBvUyMq6C2ZtN9T0ROjT', 'uri': 'spotify:track:4caBvUyMq6C2ZtN9T0ROjT', 'track_href': 'https://api.spotify.com/v1/tracks/4caBvUyMq6C2ZtN9T0ROjT', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4caBvUyMq6C2ZtN9T0ROjT', 'duration_ms': 210733, 'time_signature': 1}, {'danceability': 0.748, 'energy': 0.166, 'key': 7, 'loudness': -14.963, 'mode': 1, 'speechiness': 0.0343, 'acousticness': 0.681, 'instrumentalness': 0.482, 'liveness': 0.0813, 'valence': 0.5, 'tempo': 122.25, 'type': 'audio_features', 'id': '3Ud0Z2i2ilRkREJZkeNsye', 'uri': 'spotify:track:3Ud0Z2i2ilRkREJZkeNsye', 'track_href': 'https://api.spotify.com/v1/tracks/3Ud0Z2i2ilRkREJZkeNsye', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Ud0Z2i2ilRkREJZkeNsye', 'duratio

In [8]:
#create empty list to put input song dicts
allSongs = []

#year of first song
year = 1946

#for loop to add track info to the DataFrame
for x in range(0, totalSongs):
    
    #get song name
    songInfo = spotify.track(IDlist[x])
    songName = songInfo['name']
    
    #get song features
    trackFeatures = trackFeaturesList[x]
    danceability = trackFeatures['danceability']
    energy = trackFeatures['energy']
    loudness = trackFeatures['loudness']
    acousticness = trackFeatures['acousticness']
    tempo = trackFeatures['tempo']
    
    #create dictionary from song info
    song = {
        'year':year,
        'name':songName, 
        'danceability':danceability, 
        'energy':energy,
        'loudness':loudness,
        'acousticness':acousticness,
        'tempo':tempo  
    }
    
    #increment year with 1
    year = year + 1
    
    #add dictionary to list
    allSongs.append(song)

#change list into Pandas DataFrame
allSongsDF = pd.DataFrame(allSongs)
allSongsDF.head()

,year,name,danceability,energy,loudness,acousticness,tempo
0,1946,Prisoner of Love - Remastered - 1992,0.342,0.236,-8.276,0.762,76.747
1,1947,Near You,0.748,0.166,-14.963,0.681,122.250
2,1948,Twelfth Street Rag,0.530,0.570,-7.942,0.551,105.621
3,1949,Riders In the Sky,0.428,0.587,-5.039,0.957,126.574
4,1950,Goodnight Irene,0.335,0.255,-10.561,0.843,141.158


In [35]:

fig = go.Figure()

features = ['danceability','energy','loudness','acousticness','tempo']

for features in features : 
    tempDF = allSongsDF[features]
    fig.add_trace(go.Scatter(x=allSongsDF['year'], y=tempDF, name=features,mode='markers'))
    
    #Add the legend
    my_legend = {'x': 1, 'y': 0, 
            'bgcolor': 'rgb(237, 219, 180)', 'borderwidth': 5}
    # Update the figure
    fig.update_layout({'showlegend': True, 'legend': my_legend},
                     title="Change in audio features throughout the years (1946-2021)")
    fig.update_layout(hovermode="y")

    
sliders = [
    {'steps':[
    {'method': 'update', 'label': 'Danceability', 'args': [{'visible': [True, False, False, False, False]}]},
    {'method': 'update', 'label': 'Energy', 'args': [{'visible': [False, True, False, False, False]}]},
    {'method': 'update', 'label': 'Loudness', 'args': [{'visible': [False, False, True, False, False]}]},
    {'method': 'update', 'label': 'Acousticness', 'args': [{'visible': [False, False, False, True, False]}]},
    {'method': 'update', 'label': 'Tempo', 'args': [{'visible': [False, False, False, False, True]}]}]}]

fig.update_layout({'sliders': sliders})

fig.show()